Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [1]:
import unicodecsv

import os
print 'Home directory',os.getcwd()
## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)
    
enrollments[0]

Home directory /Users/cinq/Documents/Udacity/Intro to Data Analysis/Lesson 1


{u'account_key': u'448',
 u'cancel_date': u'2015-01-14',
 u'days_to_cancel': u'65',
 u'is_canceled': u'True',
 u'is_udacity': u'True',
 u'join_date': u'2014-11-10',
 u'status': u'canceled'}

In [2]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

with open('daily_engagement.csv', 'rb') as d:
    reader = unicodecsv.DictReader(d)
    daily_engagement = list(reader)

print daily_engagement[0]

with open('project_submissions.csv', 'rb') as p:
    reader = unicodecsv.DictReader(p)
    project_submissions = list(reader)

print project_submissions[0]



{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Fixing Data Types

In [3]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [4]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [5]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [6]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

enrollment_num_rows = len(enrollments)          
enrollment_num_unique_students = len(list(set({e['account_key'] for e in enrollments})))

engagement_num_rows = len(daily_engagement)             # Replace this with your code
engagement_num_unique_students = len(list(set({e['acct'] for e in daily_engagement})))

submission_num_rows = len(project_submissions)           # Replace this with your code
submission_num_unique_students = len(list(set({e['account_key'] for e in project_submissions})))

print 'enrollment_num_rows', enrollment_num_rows, 'enrollment_num_unique_students', enrollment_num_unique_students
print 'engagement_num_rows', engagement_num_rows, 'engagement_num_unique_students', engagement_num_unique_students
print 'submission_num_rows', submission_num_rows, 'submission_num_unique_students', submission_num_unique_students


enrollment_num_rows 1640 enrollment_num_unique_students 1302
engagement_num_rows 136240 engagement_num_unique_students 1237
submission_num_rows 3642 submission_num_unique_students 743


## Problems in the Data

In [7]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".

for d in daily_engagement:
    d['account_key'] =  d['acct']
    del d['acct']

daily_engagement[0]['account_key']

u'0'

## Missing Engagement Records

In [8]:
#####################################
#                 4                 #
#####################################
## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.



acctEnroll = set({e['account_key'] for e in enrollments})
acctEngage = set({e['account_key'] for e in daily_engagement})

studentsOnlyEnrolled = list(acctEnroll - acctEngage)

studentsEnrolledAndEngaged = list(acctEnroll - set(studentsOnlyEnrolled))

print len(studentsOnlyEnrolled)
print len(studentsEnrolledAndEngaged)


65
1237


In [9]:
import pandas as pd

studentsOnlyEnrolledList = []

for e in enrollments:
    if e['account_key'] in studentsOnlyEnrolled:
        studentsOnlyEnrolledList.append(e)
        
df = pd.DataFrame(studentsOnlyEnrolledList)        
# df

,account_key,cancel_date,days_to_cancel,is_canceled,is_udacity,join_date,status
0,1219,2014-11-12,0.0,True,False,2014-11-12,canceled
1,871,2014-11-13,0.0,True,False,2014-11-13,canceled
2,1218,2014-11-15,0.0,True,False,2014-11-15,canceled
3,654,2014-12-04,0.0,True,False,2014-12-04,canceled
4,654,2014-12-04,0.0,True,False,2014-12-04,canceled
5,654,2014-12-03,0.0,True,False,2014-12-03,canceled
6,964,2014-12-04,0.0,True,False,2014-12-04,canceled
7,817,2014-12-09,0.0,True,False,2014-12-09,canceled
8,1171,2015-01-07,0.0,True,False,2015-01-07,canceled
9,884,2015-01-08,0.0,True,False,2015-01-08,canceled


In [10]:
studentsEnrolledAndEngagedList = []

for e in enrollments:
    if e['account_key'] in studentsEnrolledAndEngaged:
        studentsEnrolledAndEngagedList.append(e)
        
df = pd.DataFrame(studentsEnrolledAndEngagedList)        
# df

,account_key,cancel_date,days_to_cancel,is_canceled,is_udacity,join_date,status
0,448,2015-01-14,65.0,True,True,2014-11-10,canceled
1,448,2014-11-10,5.0,True,True,2014-11-05,canceled
2,448,2015-01-27,0.0,True,True,2015-01-27,canceled
3,448,2014-11-10,0.0,True,True,2014-11-10,canceled
4,448,NaT,NaN,False,True,2015-03-10,current
5,448,2015-01-27,13.0,True,True,2015-01-14,canceled
6,448,2015-03-10,42.0,True,True,2015-01-27,canceled
7,448,2015-01-27,0.0,True,True,2015-01-27,canceled
8,448,2015-01-27,0.0,True,True,2015-01-27,canceled
9,700,2014-11-16,6.0,True,False,2014-11-10,canceled


## Checking for More Problem Records

In [11]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

## Tracking Down the Remaining Problems

In [12]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [13]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [14]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


## Refining the Question

In [15]:
#####################################
#                 6                 #
#####################################
## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}

for e in non_udacity_enrollments:
    if e['days_to_cancel'] > 7 or e['days_to_cancel'] == None:
        if e['account_key'] not in paid_students:
            paid_students[e['account_key']] = e['join_date']
        elif paid_students[e['account_key']] < e['join_date']:
            paid_students[e['account_key']] = e['join_date']
            

# paid_students

{u'0': datetime.datetime(2015, 1, 9, 0, 0),
 u'1': datetime.datetime(2015, 5, 6, 0, 0),
 u'10': datetime.datetime(2014, 11, 10, 0, 0),
 u'100': datetime.datetime(2015, 2, 5, 0, 0),
 u'1000': datetime.datetime(2015, 7, 11, 0, 0),
 u'1001': datetime.datetime(2015, 7, 9, 0, 0),
 u'1002': datetime.datetime(2015, 5, 7, 0, 0),
 u'1007': datetime.datetime(2015, 5, 12, 0, 0),
 u'1009': datetime.datetime(2014, 12, 5, 0, 0),
 u'101': datetime.datetime(2014, 11, 12, 0, 0),
 u'1011': datetime.datetime(2015, 4, 8, 0, 0),
 u'1013': datetime.datetime(2015, 4, 1, 0, 0),
 u'1014': datetime.datetime(2015, 7, 18, 0, 0),
 u'1015': datetime.datetime(2014, 12, 3, 0, 0),
 u'1019': datetime.datetime(2014, 11, 11, 0, 0),
 u'102': datetime.datetime(2014, 11, 10, 0, 0),
 u'1022': datetime.datetime(2015, 8, 20, 0, 0),
 u'1028': datetime.datetime(2015, 8, 21, 0, 0),
 u'1029': datetime.datetime(2015, 5, 12, 0, 0),
 u'103': datetime.datetime(2015, 1, 7, 0, 0),
 u'1030': datetime.datetime(2015, 5, 11, 0, 0),
 u'1031'

## Getting Data from First Week

In [16]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [ ]:
#####################################
#                 7                 #
#####################################
paid_engagement_in_first_week = []


for student in paid_students:
    for engage in non_udacity_engagement:
        e_account_key = engage['account_key']
        
        if(e_account_key == student) and within_one_week(paid_students[student], engage['utc_date']):
            paid_engagement_in_first_week.append(engage)

print 123
len(paid_engagement_in_first_week)

## Exploring Student Engagement

In [18]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [19]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [20]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

Mean: nan
Standard deviation: nan
Minimum:

/Users/cinq/Documents/Programs/anaconda/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/cinq/Documents/Programs/anaconda/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/cinq/Documents/Programs/anaconda/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/Users/cinq/Documents/Programs/anaconda/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/cinq/Documents/Programs/anaconda/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in doubl

ValueError: zero-size array to reduction operation minimum which has no identity

## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

## Lessons Completed in First Week

In [ ]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.